In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('data.csv')
data.keys()
data.shape

(420464, 2)

In [3]:
data["label"].value_counts()

good    344821
bad      75643
Name: label, dtype: int64

In [4]:
data = data.sample(frac=0.005, random_state=1)
data["label"].value_counts()

good    1730
bad      372
Name: label, dtype: int64

In [5]:
#finding length of url
data['LEN'] = data['url'].apply(lambda x: len(x))

In [6]:
#finding count of special characters
import re
# x = "asdfklsdf#$&^#@!"
lst = []

for i in range(len(data)):
    new = re.sub('[\w]+' ,'', data.iloc[i]['url'])
    lst.append(len(new))
#     print(i)
data['SPECIAL_CHARACTER_COUNT'] = lst

In [7]:
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

lst = []
for i in range(len(data)):
#     new = re.sub('[\w]+' ,'', data.iloc[i]['url'])
    lst.append(having_ip_address(data.iloc[i]["url"]))
#     print(i)
data['HAVING_IP_ADDR'] = lst

In [9]:
from bs4 import BeautifulSoup
import urllib
import bs4
import socket, whois

In [10]:
# checking for @ symbol
lst = []
for i in range(len(data)):
    if re.search('@', data.iloc[i]['url']):
        lst.append(1)
    else:
        lst.append(0)
        
data['HAS_AT'] = lst

In [11]:
# If shortening service been used
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0
lst = []
for i in range(len(data)):
#     print(i)
    lst.append(shortening_service(str(data.iloc[i]['url'])))
data['SHORTENING_USED'] = lst
np.unique(data['SHORTENING_USED'],return_counts=True)

(array([0, 1]), array([1959,  143]))

In [12]:
#CONTAINS HTTPS TOCKEN
def https_token(url):
    match = re.search('https://|http://', url)
    if match==None:
        return 0
    if match.start(0) == 0:
        url = url[match.end(0):]
    match = re.search('http|https', url)
    if match:
        return 0
    else:
        return 1

lst = []

for i in range(len(data)):
    lst.append(https_token(str(data.iloc[i]['url'])))

data['HTTPS_TOKEN'] = lst

In [13]:
data.to_csv('url_features1.csv', index=False)

In [14]:
reduced_data = pd.read_csv("url_features1.csv")
reduced_data.shape

(2102, 8)

In [15]:
#page rank
import sys, bs4
import urllib.request as urllib2
import ssl

gcontext = ssl._create_unverified_context()
def page_rank(url):
    temp=url
    rank =bs4.BeautifulSoup(urllib2.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+ \
    temp, context=gcontext).read(), "xml").find("REACH")['RANK']
    return rank

In [145]:
# Calculate DNS response time
import socket
import requests as req
import re
import time

def DNS_RESPONSE(url):
    try :
        url = url.split("/")
        hostname = url[0]
    except : 
        print(" issue in finding host name from url : " + url)
        return -2
    
    try :
        startTime = time.time()
        ip = socket.gethostbyname(hostname)
        endTime = time.time()
        diff = (endTime - startTime)* 1000
        return diff
    except :
        return -1

In [146]:
DNS_RESPONSE("dhrsjdyym.com/")

-1

In [147]:
DNS_RESPONSE("en.wikipedia.org/wiki/Jean-Claude_Turcotte")

0.5104541778564453

In [148]:
DNS_RESPONSE("wn.com/San_Diego_State_Aztecs")

0.6761550903320312

In [149]:
DNS_RESPONSE("pcbc.org.nz/active/libraries/joomla/cache/storage/appcloudshrdocs/Gdoc/")

316.99252128601074

In [150]:
DNS_RESPONSE("facebook.com/pages/Nichols-Supermarket-Discount-Liquors/100655924025?v=wall")

1.8956661224365234

In [151]:
DNS_RESPONSE("188.127.249.203/data/info.php")

0.041484832763671875

In [152]:
DNS_RESPONSE("bjcurio.com/js/index.htm")

258.9740753173828

In [5]:
lst = []

for i in range(len(reduced_data)):
    try:
        lst.append(DNS_RESPONSE(str(reduced_data.iloc[i]['url'])))
    except:
#         print(reduced_data.iloc[i]['url'],reduced_data.iloc[i]['label'] )
        lst.append("nan")
    if (i%50==0):
        print(i)
reduced_data['DNS_RESPONSE'] = lst

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100


In [11]:
reduced_data = reduced_data[reduced_data["DNS_RESPONSE"] != "nan"]

In [12]:
reduced_data.to_csv('url_features1.csv', index=False)

In [8]:
reduced_data.shape

(1819, 9)

In [86]:
from urllib.parse import urlparse
import time
import ssl
import urllib.request  as urllib2
import urllib

def networks(link):
    if not urlparse(link).scheme:
        url = "http://" + link
    gcontext = ssl._create_unverified_context()
    handshake_start = time.time()
    http_status_code = 400

    try : 
        stream = urllib2.urlopen(url,context=gcontext)
        http_status_code = stream.getcode()

    except urllib.error.HTTPError  as e :
            http_status_code = e.code
    
    
    if http_status_code >= 400 :
        return 0 , 0 , 0 , False , 'http' , http_status_code        
            
        
        
    new_url = stream.geturl()
    
    redirect = False
    protocol =  "http"
    
    if new_url :
        new_url = new_url.split("://")
        protocol =  new_url[0]
        del new_url[0]
        new_url = "://".join(new_url)
        
        if new_url != link:
            redirect = True

            
        
#     print(stream.info())
    handshake_end = time.time()
    handshake_time = (handshake_end - handshake_start) * 1000
#     print ('HTTP handshake time = %.5f ms' %handshake_time)
    data_start = time.time()
    data_length = len(stream.read())
    data_end = time.time()
    data_time = (data_end - data_start)*1000
#     print("data time = %.5f ms" %data_time)
#     print(data_length)
    return handshake_time, data_time, data_length , redirect , protocol , http_status_code
# networks("twitter.com/aQuariusrecOrds")

In [87]:
print(networks("www.legacy.com/obituaries/orlandosentinel/obituary.aspx?n=william-c-underwood-bill&pid=147765497"))

(2309.3605041503906, 279.5758247375488, 129049, False, 'https', 200)


In [88]:
print(networks("ant.co.id/.https/www/outlook.office365.com/MBI_SSL/w-signin/owa/auth/logon.aspx_url_replace-Current_owa/index.php?umail=YWNjb3VudGluZ0BiZXN0LWNoZW0uY28udWsN"))

(508.8222026824951, 8.513689041137695, 5937, True, 'http', 200)


In [89]:
print(networks("snip.ly/kv4hc"))

(697.4468231201172, 1.6281604766845703, 13707, False, 'http', 200)


In [90]:
print(networks("econopaginas.com/nyx37ec?eujhnelmax=fivpyok"))

(0, 0, 0, False, 'http', 404)


In [91]:
print(networks("eclecticvideo.com/cc/GD"))

(0, 0, 0, False, 'http', 404)


In [92]:
print(networks("brothermalw.ws/0b24a"))

(793.5218811035156, 0.14591217041015625, 577, False, 'http', 200)


In [93]:
print(networks("highschoolhomepages.student.com/index.php?c=37&s=59"))

(619.7001934051514, 0.09512901306152344, 0, False, 'http', 200)


In [94]:
print(networks("wikinfo.org/index.php/Texas"))

(0, 0, 0, False, 'http', 404)


In [10]:
lst_hand = []
lst_data_time=[]
lst_data_length = []

for i in range(len(reduced_data)):
    try:
        hand, data_time,data_length =  networks(str(reduced_data.iloc[i]['url']))
        lst_hand.append(hand)
        lst_data_time.append(data_time)
        lst_data_length.append(data_length)
    except:
        lst_hand.append("nan")
        lst_data_time.append("nan")
        lst_data_length.append("nan")
    if (i%50==0):
        print(i)
reduced_data["HTTP_HANDSHAKE"] = lst_hand
reduced_data["DATA_TIME"] = lst_data_time
reduced_data["DATA_LENGTH"] = lst_data_length

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800


In [108]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
data = pd.read_csv("preprocessed.csv")

y= data[""]
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(X, y)
clf.predict(X[:2, :])

clf.predict_proba(X[:2, :]) 


clf.score(X, y)

KeyError: ''

In [187]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("preprocessed.csv")
data = data.dropna()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
scaler = StandardScaler()

y = data.pop('label')

data = data.drop("url_name",axis=1)
X = data
X = scaler.fit_transform(X)

y = le.fit_transform(y)
# np.unique(y,return_counts=True)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4)
# print(X_train.shape)
# print(X_test.shape)
# 
clf = LogisticRegression(class_weight={0:7, 1:1}).fit(X_train, y_train)
# clf.predict(X[:2, :])

# clf.predict_proba(X[:2, :]) 


print(clf.score(X_test, y_test))
pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,pred))

0.7090395480225988
             precision    recall  f1-score   support

          0       0.24      0.86      0.38        73
          1       0.98      0.69      0.81       635

avg / total       0.90      0.71      0.77       708

[[ 63  10]
 [196 439]]
